In [ ]:
import torch
import numpy as np

# --------------------------------------------------------------------
# codes
# --------------------------------------------------------------------
# sorting 후 한 카드에 2개 인식된거 1개만 남겨둠
def preprocessing_arr(array):
  array.sort()
  raw_len = len(array) - 1
  idx = 1
  while idx <= raw_len:
    if array[idx - 1] == array[idx]:
      del array[idx]
      raw_len -= 1
    idx += 1
  return array

# 패에 같은 숫자인 카드 있는지 확인
# nothing = 0, pair = 1, two pair = 2, triple = 3, four = 4, full = 5
def isNumSame(array):
  flag = 0
  is_pair = False
  is_triple = False
  temp = -1
  temp_arr = []
  for i in range(5):
    temp_arr.append(array[i]%13)

  for i in range(5):
    # 같은 숫자인 카드 있는지 확인
    for j in range(i + 1, 5):
      # pair 확인
      if temp_arr[i] == temp_arr[j]:
        # pair일 경우 같은거 더 있는지 확인
        for k in range(j + 1, 5):
          # triple 확인
          if temp_arr[j] == temp_arr[k]:
            # triple일 경우 같은거 더 있는지 확인
            for l in range(k + 1, 5):
              # fourcard 확인
              if temp_arr[k] == temp_arr[l]:
                # fourcard
                return 4
            # 같은게 3개밖에 없으니 일단 triple
            is_triple = True
            flag = 3
            # 숫자 저장
            temp = temp_arr[i]
        # 처음으로 나온 pair이면서 triple 아닐 경우
        if temp == -1:
          # one pair
          flag = 1
          is_pair = True
          # 숫자 저장
          temp = temp_arr[j]

        # 현재 triple일 경우
        elif is_triple and temp == temp_arr[i]:
          # 숫자 저장
          temp = temp_arr[j]
          # 이전에 pair가 나왔었다면
          if is_pair:
            # full house
            return 5

        else:
          # 이전에 triple이 나왔고 현재 pair인 경우
          if is_triple:
            # full house
            return 5
          # 두번째 pair인 경우
          # two pair
          return 2
  return flag

# 숫자가 연속되어있는지 확인
# nothing = 0, straight = 1, back straight = 2, mountain = 3
def isSeries(array):
  temp_arr = []
  for i in range(5):
    temp_arr.append(array[i]%13)

  temp_arr.sort()
  # Ace가 있을 경우
  if temp_arr[0] == 0:
    # king이 있을 경우
    if temp_arr[1] == 1:
      # mountain 체크
      for i in range(2, 5):
        # 중간에 하나라도 아니면 아무것도 아님
        if temp_arr[i-1] != temp_arr[i] - 1:
          return 0
      # 전부 맞으면 mountain
      return 3

    # 5가 있을 경우
    elif temp_arr[1] == 9:
      for i in range(2, 5):
        # 중간에 하나라도 아니면 아무것도 아님
        if temp_arr[i-1] != temp_arr[i] - 1:
            return 0
      # 전부 맞으면 back straight
      return 2
    # 둘다 없으면 아님
    return 0

  # straight인지 확인
  else:
    for i in range(1, 5):
      # 하나라도 연속되어있지 않으면 아무것도 아님
      if temp_arr[i-1] != temp_arr[i] - 1:
        return 0
    # 모두 연속되어있고, Ace가 없으면
    return 1

# 같은 문양인지 확인
# nothing = 0, flush = 1
def isFlush(array):
  for i in range(4):
    if int(array[i]/13) != int(array[i+1]/13):
      return 0
  return 1

def app(array):
  labels = ["Nothing", "One Pair", "Two Pair", "Triple", "Straight",
            "Backstraight", "Mountain", "Flush", "Full House", "Four Card",
            "Straight Flush", "Backstraight Flush", "Royal Straight Flush"]
  result = 0
  # preprocessing(sorting 후 중복된 값 제거)
  array = preprocessing_arr(array)
  print(array)

  # 패에 같은 숫자인 카드 있는지 확인
  # nothing = 0, pair = 1, two pair = 2, triple = 3, four = 4, full = 5
  is_num_same = isNumSame(array)
  #print("nothing = 0, pair = 1, two pair = 2, triple = 3, four = 4, full = 5")
  #print("is_num_same: ", is_num_same)

  # one pair
  if is_num_same == 1:
    return labels[1]
  # two pair
  elif is_num_same == 2:
    return labels[2]
  # triple
  elif is_num_same == 3:
    return labels[3]
  # four card
  elif is_num_same == 4:
    return labels[9]
  # full house
  elif is_num_same == 5:
    return labels[8]

  # 숫자가 연속되어있는지 확인
  # nothing = 0, straight = 1, back straight = 2, mountain = 3
  is_series = isSeries(array)
  #print("nothing = 0, straight = 1, back straight = 2, mountain = 3")
  #print("is_series: ", is_series)
  # straight
  if is_series == 1:
    result = 4
  # backstraight
  elif is_series == 2:
    result = 5
  # mountain
  elif is_series == 3:
    result = 6

  # 같은 문양인지 확인
  # nothing = 0, flush = 1
  is_flush = isFlush(array)
  #print("Flush?: ", is_flush)

  if is_flush:
    # straight flush
    if is_series == 1:
      return labels[10]
    # back straight flush
    elif is_series == 2:
      return labels[11]
    # royal straight flush
    elif is_series == 3:
      return labels[12]
    # flush
    else:
      return labels[7]

  return labels[result]

----------------------------------------------------------
sample0
[0, 1, 2, 3, 4]
Royal Straight Flush
----------------------------------------------------------
sample1
[26, 35, 36, 37, 38]
Backstraight Flush
----------------------------------------------------------
sample2
[1, 2, 3, 4, 5]
Straight Flush
----------------------------------------------------------
sample3
[11, 24, 37, 50, 51]
Four Card
----------------------------------------------------------
sample4
[10, 23, 31, 36, 44]
Full House
----------------------------------------------------------
sample5
[2, 4, 7, 9, 11]
Flush
----------------------------------------------------------
sample6
[2, 13, 27, 30, 42]
Mountain
----------------------------------------------------------
sample7
[12, 13, 36, 37, 48]
Backstraight
----------------------------------------------------------
sample8
[7, 22, 24, 36, 47]
Straight
----------------------------------------------------------
sample9
[7, 20, 24, 30, 46]
Triple
-----------------

In [ ]:
# --------------------------------------------------------------------
# test datas
# --------------------------------------------------------------------
# 로얄 스트레이트 플러쉬
sample0 = [0, 0, 1, 1, 2, 2, 3, 3, 4, 4]
# 백스트레이트 플러쉬
sample1 = [38, 38, 36, 36, 35, 35, 37, 37, 26, 26]
# 스트레이트 플러쉬
sample2 = [1, 1, 2, 3, 3, 4, 5]
# 포카드
sample3 = [11, 11, 24, 24, 37, 37, 50, 50, 51, 51]
# 풀하우스
sample4 = [10, 10, 36, 36, 23, 23, 31, 31, 44, 44]
# 플러쉬
sample5 = [4, 4, 2, 2, 9, 9, 7, 7, 11, 11]
# 마운틴
sample6 = [2, 2, 13, 13, 30, 30, 27, 27, 42, 42]
# 백스트레이트
sample7 = [48, 48, 36, 36, 12, 12, 13, 13, 37]
# 스트레이트
sample8 = [24, 24, 36, 47, 47, 22, 22, 7, 7]
# 트리플
sample9 = [24, 24, 46, 46, 20, 30, 30, 7, 7]
# 투페어
sample10 = [22, 28, 19, 19, 48, 6, 6]
# 원페어
sample11 = [0, 3, 10, 32, 45, 45]
# 없음
sample12 = [3, 3, 36, 36, 18, 18, 33, 33, 48, 48]
# --------------------------------------------------------------------

# --------------------------------------------------------------------
# testing
# --------------------------------------------------------------------
print("----------------------------------------------------------")
print("sample0")
print(app(sample0)) # Royal Straight Flush
print("----------------------------------------------------------")
print("sample1")
print(app(sample1)) # Back Straight Flush
print("----------------------------------------------------------")
print("sample2")
print(app(sample2)) # Straight Flush
print("----------------------------------------------------------")
print("sample3")
print(app(sample3)) # Four Card
print("----------------------------------------------------------")
print("sample4")
print(app(sample4)) # Full house
print("----------------------------------------------------------")
print("sample5")
print(app(sample5)) # Flush
print("----------------------------------------------------------")
print("sample6")
print(app(sample6)) # Mountain
print("----------------------------------------------------------")
print("sample7")
print(app(sample7)) # Backstraight
print("----------------------------------------------------------")
print("sample8")
print(app(sample8)) # Straight
print("----------------------------------------------------------")
print("sample9")
print(app(sample9)) # Triple
print("----------------------------------------------------------")
print("sample10")
print(app(sample10)) # Two Pair
print("----------------------------------------------------------")
print("sample11")
print(app(sample11)) # One Pair
print("----------------------------------------------------------")
print("sample12")
print(app(sample12)) # Nothing
print("----------------------------------------------------------")
# --------------------------------------------------------------------